In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import super_res
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 8192)
model = super_res.AudioTFILM(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Tensor("generator/downsc_conv0/Reshape_2:0", shape=(?, 4096, 128), dtype=float32)
Tensor("generator/downsc_conv1/Reshape_2:0", shape=(?, 2048, 256), dtype=float32)
Tensor("generator/downsc_conv2/Reshape_2:0", shape=(?, 1024, 512), dtype=float32)
Tensor("generator/downsc_conv3/Reshape_2:0", shape=(?, 512, 512), dtype=float32)
Tensor("generator/bottleneck_conv/Reshape_2:0", shape=(?, 256, 512), dtype=float32)

Tensor("generator/upsc_conv3/concat:0", shape=(?, 512, 1024), dtype=float32)
Tensor("generator/upsc_conv2/concat:0", shape=(?, 1024, 1024), dtype=float32)
Tensor("generator/upsc_conv1/concat:0", shape=(?, 2048, 512), dtype=float32)
Tensor("generator/upsc_conv0/concat:0", shape=(?, 4096, 256), dtype=float32)
Tensor("generator/lastconv/subpixel/transpose_1:0", shape=(?, 8192, 1), dtype=float32)


In [5]:
model.logits

<tf.Tensor 'generator/add:0' shape=(?, 8192, 1) dtype=float32>

In [6]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_4:0' shape=(?, 1) dtype=float32>

In [7]:
snr = enhancement.loss.snr(model.logits, partitioned_x)
sdr = enhancement.loss.sdr(model.logits, partitioned_x)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [19]:
y, sr = malaya_speech.load('../speech/podcast/SKOLAR.wav', sr = 44100)
y = y[:sr * 10]

In [20]:
import IPython.display as ipd

ipd.Audio(y, rate = sr)

In [21]:
def downsample(y, sr, down_sr):
    y_ = malaya_speech.resample(y, sr, down_sr)
    return malaya_speech.resample(y_, down_sr, sr)

In [ ]:
selected_sr = [4000, 6000, 8000, 16000]

In [24]:
y_ = downsample(y, sr, 8000)

In [25]:
ipd.Audio(y_, rate = sr)

In [13]:
%%time

y_ = sess.run(model.logits, feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

CPU times: user 1min 43s, sys: 5.48 s, total: 1min 49s
Wall time: 18 s


(32, 8192, 1)

In [14]:
y_.reshape((-1)).shape

(262144,)

In [15]:
y.shape

(258867,)

In [16]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [17]:
y_, snr_, sdr_ = sess.run([model.logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(32, 8192, 1)

In [18]:
snr_, sdr_

((0.0010017452, 35.37982), -3.8146964e-06)

In [19]:
saver = tf.train.Saver()

In [20]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [21]:
!ls -lh test

total 534568
-rw-r--r--  1 huseinzolkepli  staff    77B Jan 12 17:41 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   260M Jan 12 17:41 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff   2.1K Jan 12 17:41 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   782K Jan 12 17:41 model.ckpt.meta


In [22]:
tf.trainable_variables()

[<tf.Variable 'generator/downsc_conv0/conv1d/kernel:0' shape=(65, 1, 128) dtype=float32>,
 <tf.Variable 'generator/downsc_conv0/conv1d/bias:0' shape=(128,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv0/lstm/kernel:0' shape=(128, 512) dtype=float32>,
 <tf.Variable 'generator/downsc_conv0/lstm/recurrent_kernel:0' shape=(128, 512) dtype=float32>,
 <tf.Variable 'generator/downsc_conv0/lstm/bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/conv1d_1/kernel:0' shape=(33, 128, 256) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/conv1d_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/lstm_1/kernel:0' shape=(256, 1024) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/lstm_1/recurrent_kernel:0' shape=(256, 1024) dtype=float32>,
 <tf.Variable 'generator/downsc_conv1/lstm_1/bias:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'generator/downsc_conv2/conv1d_2/kernel:0' shape=(17, 256, 512) dtype=float32>,
 <tf.Variable 'generato

In [23]:
!rm -rf test